In [2]:
# !wget https://object.pouta.csc.fi/OPUS-KDE4/v2/raw/en.zip
# !wget https://object.pouta.csc.fi/OPUS-KDE4/v2/raw/ms.zip
# !unzip -o en.zip
# !unzip -o ms.zip

In [4]:
# !wget wget https://object.pouta.csc.fi/OPUS-KDE4/v2/xml/en-ms.xml.gz
# !gzip -d en-ms.xml.gz

In [5]:
from bs4 import BeautifulSoup
from tqdm import tqdm

with open('en-ms.xml') as fopen:
    xml = fopen.read().split('\n')
    
mapping, ids = {}, []
enable = False
for row in tqdm(xml):
    if '<linkGrp targType' in row:
        s = BeautifulSoup(row)
        r = 'KDE4/raw/' + s.findAll('linkgrp')[0].get('fromdoc').replace('.gz','')
        l = 'KDE4/raw/' + s.findAll('linkgrp')[0].get('todoc').replace('.gz','')
        enable = True
    if '<link id' in row and enable:
        s = BeautifulSoup(row)
        t = s.findAll('link')[0].get('xtargets')
        t = list(filter(None, t.split(';')))
        if len(t) != 2:
            continue
        ids.append([t[0].split(), t[1].split()])
    if '</linkGrp>' in row:
        label = f'{l} <> {r}'
        mapping[label] = ids
        enable = False
        ids = []

100%|██████████| 103657/103657 [00:24<00:00, 4261.41it/s]


In [6]:
mapping[label]

[[['s2'], ['s2']],
 [['s4'], ['s4']],
 [['s6'], ['s6']],
 [['s8'], ['s8']],
 [['s13'], ['s13']]]

In [7]:
results = []
for file in tqdm(mapping.keys()):
    l, r = file.split(' <> ')
    ids = mapping[file]
    try:
        with open(l) as fopen:
            left = fopen.read()
        soup_left = BeautifulSoup(left, 'html.parser')
        with open(r) as fopen:
            right = fopen.read()
        soup_right = BeautifulSoup(right, 'html.parser')
        left_dict = {}
        for s in soup_left.find_all('s'):
            i = s.get('id')
            text = s.get_text()
            left_dict[i] = text
        right_dict = {}
        for s in soup_right.find_all('s'):
            i = s.get('id')
            text = s.get_text()
            right_dict[i] = text
        combined = []
        for i in ids:
            try:
                r = ' '.join([right_dict[k] for k in i[0]])
                l = ' '.join([left_dict[k] for k in i[1]])
                combined.append((l, r))
            except:
                pass
        results.extend(combined)
    except Exception as e:
        print(e)
        pass

 95%|█████████▌| 575/604 [00:12<00:01, 27.42it/s] 

[Errno 2] No such file or directory: 'KDE4/raw/ms/messages/playground-multimedia/kdetv.xml'


100%|██████████| 604/604 [00:13<00:00, 45.25it/s]


In [8]:
len(results)

87184

In [11]:
results[:10]

[('Tambah Feed ke Akregator', 'Add Feed to Akregator'),
 ('Tambah Feeds ke Akregator', 'Add Feeds to Akregator'),
 ('Tambah Semua Feed Yang Ditemui ke Akregator',
  'Add All Found Feeds to Akregator'),
 ('Feeds yang diimport', 'Imported Feeds'),
 ('& Automuat semula', '& Auto Refresh'),
 ('Tiada', 'None'),
 ('Setiap 15 saat', 'Every 15 Seconds'),
 ('Setiap 30 saat', 'Every 30 Seconds'),
 ('Setiap Minit', 'Every Minute'),
 ('Setiap 5 Minit', 'Every 5 Minutes')]

In [9]:
import json

with open('kde4-ms-en.json', 'w') as fopen:
    json.dump(results, fopen)

In [10]:
!rm -rf KDE4
!rm -rf ms.zip en.zip en-ms.xml.gz en-ms.xml